<a href="https://colab.research.google.com/github/asantosf/TransPort/blob/main/Datos_externos_archivos_locales%2C_Drive%2C_Hojas_de_c%C3%A1lculo_y_Cloud_Storag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Incisos

In [20]:
!pip install pyspark

In [2]:
!pip install findspark

In [3]:
import findspark
findspark.init()

import pandas as pd
import pyspark

In [4]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [5]:
person = spark.read.option('delimiter', ';').option('header','true').csv('Person.Person.csv')
product = spark.read.option('delimiter', ';').option('header','true').csv('Production.Product.csv')
customer = spark.read.option('delimiter', ';').option('header','true').csv('Sales.Customer.csv')
salesOrderDetail = spark.read.option('delimiter', ';').option('header','true').csv('Sales.SalesOrderDetail.csv')
salesOrderHeader = spark.read.option('delimiter', ';').option('header','true').csv('Sales.SalesOrderHeader.csv')
territory = spark.read.option('delimiter', ';').option('header','true').csv('Sales.SalesTerritory.csv')
salesOrderHeader = spark.read.option("delimiter", ";").option("header", "true").csv('Sales.SalesOrderHeader.csv')
territory = spark.read.option('delimiter',';').option('header','true').csv('Sales.SalesTerritory.csv')
secuenciales = spark.read.option('delimiter', ',').option('header','true').csv('Secuenciales.csv')

In [208]:
salesOrderHeader.join(territory, salesOrderHeader['TerritoryID'] == territory['TerritoryID']).select(
    date_format('OrderDate', 'yyyy-MM').alias('Mes'), 'Group',
    # Cantidades
    when(col('Status') == 1, 1).otherwise(0).alias('status_1'),
    when(col('Status') == 2, 1).otherwise(0).alias('status_2'),
    when(col('Status') == 3, 1).otherwise(0).alias('status_3'),
    when(col('Status') == 4, 1).otherwise(0).alias('status_4'),
    when(col('Status') == 5, 1).otherwise(0).alias('status_5'),
    when(col('Status') == 6, 1).otherwise(0).alias('status_6'),
    # Montos
    when(col('Status') == 1, salesOrderHeader['TotalDue'].cast('float')).otherwise(0).alias('monto_1'),
    when(col('Status') == 2, salesOrderHeader['TotalDue'].cast('float')).otherwise(0).alias('monto_2'),
    when(col('Status') == 3, salesOrderHeader['TotalDue'].cast('float')).otherwise(0).alias('monto_3'),
    when(col('Status') == 4, salesOrderHeader['TotalDue'].cast('float')).otherwise(0).alias('monto_4'),
    when(col('Status') == 5, salesOrderHeader['TotalDue'].cast('float')).otherwise(0).alias('monto_5'),
    when(col('Status') == 6, salesOrderHeader['TotalDue'].cast('float')).otherwise(0).alias('monto_6')
    ).groupBy(
        'Mes', 'Group'
    ).pivot('status_1').agg(
        sum('status_1').alias('TrProceso'),
        sum('status_2').alias('TrAprobadas'),
        sum('status_3').alias('TrAtrasadas'),
        sum('status_4').alias('TrRechazadas'),
        sum('status_5').alias('TrEnviadas'),
        sum('status_6').alias('TrCanceladas'),
        sum('monto_1').alias('MntProceso'),
        sum('monto_2').alias('MntAprobadas'),
        sum('monto_3').alias('MntAtrasadas'),
        sum('monto_4').alias('MntRechazadas'),
        sum('monto_5').alias('MntEnviadas'),
        sum('monto_6').alias('MntCanceladas')
    ).orderBy('Mes', 'Group').show()

+-------+-------------+-----------+-------------+-------------+--------------+------------+--------------+------------+--------------+--------------+---------------+------------------+---------------+
|    Mes|        Group|0_TrProceso|0_TrAprobadas|0_TrAtrasadas|0_TrRechazadas|0_TrEnviadas|0_TrCanceladas|0_MntProceso|0_MntAprobadas|0_MntAtrasadas|0_MntRechazadas|     0_MntEnviadas|0_MntCanceladas|
+-------+-------------+-----------+-------------+-------------+--------------+------------+--------------+------------+--------------+--------------+---------------+------------------+---------------+
|2011-05|       Europe|          0|            0|            0|             0|           1|             0|         0.0|           0.0|           0.0|            0.0| 3756.989013671875|            0.0|
|2011-05|North America|          0|            0|            0|             0|          41|             0|         0.0|           0.0|           0.0|            0.0| 559506.9689331055|            

In [209]:
inciso2_j1 = salesOrderDetail.join(
                salesOrderHeader, salesOrderDetail['SalesOrderID'] == salesOrderHeader['SalesOrderID']
                ).select(
                    date_format('OrderDate', 'yyyy-MM').alias('Mes'), 'ProductID', 'OrderQty'
                ).where(
                    salesOrderHeader['Status'] == 5
                )

productos = product.join(inciso2_j1, product['ProductID'] == inciso2_j1['ProductID']).select(
                    'Mes', product['Name'].alias('ProductName'), 'OrderQty'
                ).groupby(
                    'Mes', 'ProductName'
                ).agg(
                    sum('OrderQty').alias('ProductQty')
                )

rn = Window.partitionBy('Mes').orderBy('Mes',col('ProductQty').desc())
rn_productos = productos.withColumn('row_no', row_number().over(rn))

In [210]:
topProductos = rn_productos.select(
        'Mes',
        when(col('row_no') == 1, rn_productos['ProductQty']).otherwise(0).alias('Producto_N1'),
        when(col('row_no') == 2, rn_productos['ProductQty']).otherwise(0).alias('Producto_N2'),
        when(col('row_no') == 3, rn_productos['ProductQty']).otherwise(0).alias('Producto_N3')
).where(
    col('row_no').cast('int') <= 3
).groupBy(
    'Mes'
).agg(
    sum(col('Producto_N1').cast('int')).alias('Producto_N1'),
    sum(col('Producto_N2').cast('int')).alias('Producto_N2'),
    sum(col('Producto_N3').cast('int')).alias('Producto_N3')
)

In [212]:
inciso2_j2 = salesOrderDetail.join(salesOrderHeader, salesOrderDetail['SalesOrderID'] == salesOrderHeader['SalesOrderID']
                                  ).select(
                                    date_format('OrderDate', 'yyyy-MM').alias('Mes'), 'CustomerID', 'TotalDue'
                                  ).filter(col('Status') == 5)

tableCus = customer.join(inciso2_j2, customer['CustomerID'] == inciso2_j2['CustomerID']).select(
    'Mes', inciso2_j2['CustomerID'], 'TotalDue'
).groupBy(
    'Mes', 'CustomerID'
).agg(
    sum('TotalDue').alias('Total')
)

row_num_1 = Window.partitionBy('Mes').orderBy('Mes', col('Total').desc())
tblGen = tableCus.withColumn('row_num', row_number().over(row_num_1))

In [213]:
topClientes = tblGen.select(
    'Mes',
    when(col('row_num') == 1, tblGen['Total']).alias('Cliente_N1'),
    when(col('row_num') == 2, tblGen['Total']).alias('Cliente_N2'),
    when(col('row_num') == 3, tblGen['Total']).alias('Cliente_N3')
).where(
    col('row_num').cast('int') <= 3
).groupBy(
    'Mes'
).agg(
    sum('Cliente_N1').alias('Cliente_N1'),
    sum('Cliente_N2').alias('Cliente_N2'),
    sum('Cliente_N3').alias('Cliente_N3')
)

In [214]:
topProductos.join(topClientes, topProductos['Mes'] == topClientes['Mes']).select(
    topProductos['Mes'], 'Producto_N1', 'Producto_N2', 'Producto_N3', 'Cliente_N1', 'Cliente_N2', 'Cliente_N3'
).show()

+-------+-----------+-----------+-----------+--------------------+--------------------+------------------+
|    Mes|Producto_N1|Producto_N2|Producto_N3|          Cliente_N1|          Cliente_N2|        Cliente_N3|
+-------+-----------+-----------+-----------+--------------------+--------------------+------------------+
|2011-05|         49|         46|         44|        1214147.7488|  1174143.9756999994| 804115.3322999998|
|2011-06|         28|         23|         21|           3953.9884|           3953.9884|         3953.9884|
|2011-07|        134|        114|        103|   2884209.041700001|        2869162.6238|1606959.9952000005|
|2011-08|        167|        137|        113|        2255571.7228|  1921769.8647000003|1881760.2210000013|
|2011-09|         27|         23|         23|           3953.9884|           3953.9884|         3953.9884|
|2011-10|        240|        239|        224|   2846244.397999999|        2655280.3025|2600539.4975999985|
|2011-11|         40|         35|    

In [215]:
maxDate = salesOrderHeader.select(max('OrderDate').alias('maxDate'))

a = salesOrderHeader.join(territory, territory['TerritoryID'] == salesOrderHeader['TerritoryID']
                                  ).select(
                                      date_format('OrderDate', 'yyyy-MM').alias('Mes'), col('Group').alias('TerritoryGroup'),
                                      col('SubTotal').cast('float').alias('SubTotal'), 'OrderDate'
                                  ).join(
                                      maxDate, salesOrderHeader['OrderDate'] >= add_months(maxDate['maxDate'],-5)
                                  ).groupBy(
                                      'Mes', 'TerritoryGroup'
                                  ).agg(
                                      avg('SubTotal').alias('GroupMonthAvg')
                                  )

b = salesOrderHeader.join(maxDate, salesOrderHeader['OrderDate'] >= add_months(maxDate['maxDate'],-5)
).select(
    date_format('OrderDate', 'yyyy-MM').alias('Mes'), 'SubTotal'
).groupBy(
    'Mes'
).agg(
    avg('SubTotal').alias('MonthAvg')
)

In [216]:
finalResult = a.join(b, a['Mes'] == b['Mes']).select(
    a.Mes,
    a.TerritoryGroup,
    a.GroupMonthAvg,
    b.MonthAvg,
    (a['GroupMonthAvg'] - b['MonthAvg']).alias('Diff'),
    when(col('GroupMonthAvg').cast('float') > col('MonthAvg').cast('float'), 'Above').otherwise('Below').alias('Indicator')
).orderBy(col('Mes').desc()).show()

+-------+--------------+------------------+------------------+-------------------+---------+
|    Mes|TerritoryGroup|     GroupMonthAvg|          MonthAvg|               Diff|Indicator|
+-------+--------------+------------------+------------------+-------------------+---------+
|2014-06| North America| 52.73099850439982|52.189392971246086|  0.541605533153735|    Above|
|2014-06|       Pacific|53.645833715086894|52.189392971246086| 1.4564407438408082|    Above|
|2014-06|        Europe| 49.72072738842531|52.189392971246086| -2.468665582820776|    Below|
|2014-05|        Europe| 1930.117516871531| 2225.912471713053|  -295.794954841522|    Below|
|2014-05| North America| 2754.944705779644| 2225.912471713053|  529.0322340665912|    Above|
|2014-05|       Pacific|1391.9935544619589| 2225.912471713053| -833.9189172510942|    Below|
|2014-04| North America| 784.6804316441218| 849.7276231678421| -65.04719152372036|    Below|
|2014-04|       Pacific| 918.0368452882767| 849.7276231678421|  68.309

In [219]:
codigoAnt = 0
count = 0
result = []

for row in secuenciales.collect():
  if codigoAnt == 0 or codigoAnt == row.codigo:
    count+=1
  else:
    codigoAnt = row.codigo
    count = 0

  if count == 3:
    result.append(row.codigo)

print(result)

['3607']


In [17]:
row_num5 = Window.partitionBy('CustomerID').orderBy(col('OrderDate').desc())
ultimasTresCompras = salesOrderHeader.withColumn('row_num', row_number().over(row_num5)).select(
    date_format('OrderDate', 'yyyy-MM-dd').alias('Mes'), 'CustomerID', 'row_num',
    when(col('row_num') == 1, date_format('OrderDate', 'yyyy-MM-dd')).otherwise('-').alias('FechaUltCompra'),
    when(col('row_num') == 2, date_format('OrderDate', 'yyyy-MM-dd')).otherwise('-').alias('FechaPenCompra'),
    when(col('row_num') == 3, date_format('OrderDate', 'yyyy-MM-dd')).otherwise('-').alias('FechaAntCompra')
).where(
    salesOrderHeader['Status'] == 5
).where(
    col('row_num') <= 3
).groupBy(
    'CustomerID'
).agg(
    max('FechaUltCompra').alias('FechaUltCompra'),
    max('FechaPenCompra').alias('FechaPenCompra'),
    max('FechaAntCompra').alias('FechaAntCompra')
)

In [19]:
ultimasTresCompras.join(
    customer, customer['CustomerID'] == ultimasTresCompras['CustomerID']
).join(
    person, customer['CustomerID'] == person['BusinessEntityID']
).select(
    date_format('FechaUltCompra','yyyy-MM').alias('Mes'),
    concat_ws(' ', person['FirstName'], person['LastName']).alias('FullName'),
    'FechaUltCompra',
    coalesce(datediff('FechaUltCompra', 'FechaPenCompra'), lit(0)).alias('DiasUC_PC'),
    coalesce(datediff('FechaPenCompra', 'FechaAntCompra'), lit(0)).alias('DiasPC_APC')
).orderBy(
    col('Mes').desc(), 'FullName'
).show()

+-------+----------------+--------------+---------+----------+
|    Mes|        FullName|FechaUltCompra|DiasUC_PC|DiasPC_APC|
+-------+----------------+--------------+---------+----------+
|2014-06|   Aaron Collins|    2014-06-28|       49|        33|
|2014-06|      Abby Madan|    2014-06-04|      113|         0|
|2014-06|  Abigail Barnes|    2014-06-04|       50|       255|
|2014-06| Abigail Johnson|    2014-06-22|      118|         1|
|2014-06|   Abigail Jones|    2014-06-28|      198|        69|
|2014-06|  Abigail Martin|    2014-06-18|       17|       299|
|2014-06|Abigail Martinez|    2014-06-15|      323|         0|
|2014-06|  Abigail Miller|    2014-06-08|      327|         0|
|2014-06|   Abigail Perry|    2014-06-26|        0|         0|
|2014-06|      Adam Lopez|    2014-06-10|        0|         0|
|2014-06|      Adam Scott|    2014-06-02|        0|         0|
|2014-06|         Alan Hu|    2014-06-09|       59|        79|
|2014-06|       Alan Yang|    2014-06-27|      360|    

# Curso Spark

In [21]:
emp = [(1, "AAA", "dept1", 1000),
    (2, "BBB", "dept1", 1100),
    (3, "CCC", "dept1", 3000),
    (4, "DDD", "dept1", 1500),
    (5, "EEE", "dept2", 8000),
    (6, "FFF", "dept2", 7200),
    (7, "GGG", "dept3", 7100),
    (8, "HHH", "dept3", 3700),
    (9, "III", "dept3", 4500),
    (10, "JJJ", "dept5", 3400)]

dept = [("dept1", "Department - 1"),
        ("dept2", "Department - 2"),
        ("dept3", "Department - 3"),
        ("dept4", "Department - 4")

       ]

df = spark.createDataFrame(emp, ["id", "name", "dept", "salary"])

deptdf = spark.createDataFrame(dept, ["id", "name"])

In [22]:
df.show()

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
|  2| BBB|dept1|  1100|
|  3| CCC|dept1|  3000|
|  4| DDD|dept1|  1500|
|  5| EEE|dept2|  8000|
|  6| FFF|dept2|  7200|
|  7| GGG|dept3|  7100|
|  8| HHH|dept3|  3700|
|  9| III|dept3|  4500|
| 10| JJJ|dept5|  3400|
+---+----+-----+------+



In [23]:
# Register DataFrame as Temporary Table
df.createOrReplaceTempView("temp_table")

# Execute SQL-Like query.
spark.sql("select * from temp_table where id = 1").show()

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
+---+----+-----+------+



In [37]:
spark.sql("select * from temp_table").select(
    'name', 'dept'
).filter(col('salary').cast('float') <= 1500).show()

+----+-----+
|name| dept|
+----+-----+
| AAA|dept1|
| BBB|dept1|
| DDD|dept1|
+----+-----+



In [112]:
spark.sql("select count(*) from temp_table where salary >= 1500").show()

+--------+
|count(1)|
+--------+
|       8|
+--------+



In [114]:
# FUNDAMENTOS DE APACHE SPARK: FUNCIONES AVANZADAS

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import broadcast
from pyspark.sql.types import *

emp = [(1, "AAA", "dept1", 1000),
    (2, "BBB", "dept1", 1100),
    (3, "CCC", "dept1", 3000),
    (4, "DDD", "dept1", 1500),
    (5, "EEE", "dept2", 8000),
    (6, "FFF", "dept2", 7200),
    (7, "GGG", "dept3", 7100),
    (None, None, None, 7500),
    (9, "III", None, 4500),
    (10, None, "dept5", 2500)]

dept = [("dept1", "Department - 1"),
        ("dept2", "Department - 2"),
        ("dept3", "Department - 3"),
        ("dept4", "Department - 4")
       ]

df = spark.createDataFrame(emp, ["id", "name", "dept", "salary"])
deptdf = spark.createDataFrame(dept, ["id", "name"])

# Create Temp Tables
df.createOrReplaceTempView("empdf")
deptdf.createOrReplaceTempView("deptdf")

# Save as HIVE tables. Para poder recuperarlas en caso se cierre sesión.
df.write.saveAsTable("hive_empdf", mode = "overwrite")
deptdf.write.saveAsTable("hive_deptdf", mode = "overwrite")

In [130]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 50 * 1024 * 1024)
join_df = df.join(broadcast(deptdf), df["id"] == deptdf["id"])

In [131]:
df.cache()
df.count()
print("Memory Used : {0}".format(df.storageLevel.useMemory))
print("Disk Used : {0}".format(df.storageLevel.useDisk))

Memory Used : True
Disk Used : True


In [132]:
from pyspark.storagelevel import StorageLevel

In [133]:
deptdf.persist(StorageLevel.MEMORY_ONLY)
deptdf.count()
print("Memory Used : {0}".format(df.storageLevel.useMemory))
print("Disk Used : {0}".format(df.storageLevel.useDisk))

Memory Used : True
Disk Used : True


In [134]:
df.unpersist()

DataFrame[id: bigint, name: string, dept: string, salary: bigint]

In [139]:
from pyspark.sql.functions import expr

# Intentemos categorizar el salario en Bajo, Medio y Alto según la categorización a continuación.

# 0-2000: salario_bajo
# 2001 - 5000: mid_salary
#> 5001: high_salary

cond = """case when salary > 5000 then 'high_salary'
               else case when salary > 2000 then 'mid_salary'
                    else case when salary > 0 then 'low_salary'
                         else 'invalid_salary'
                              end
                         end
                end as salary_level"""

newdf = df.withColumn("salary_level", expr(cond))
newdf.show()

+----+----+-----+------+------------+
|  id|name| dept|salary|salary_level|
+----+----+-----+------+------------+
|   1| AAA|dept1|  1000|  low_salary|
|   2| BBB|dept1|  1100|  low_salary|
|   3| CCC|dept1|  3000|  mid_salary|
|   4| DDD|dept1|  1500|  low_salary|
|   5| EEE|dept2|  8000| high_salary|
|   6| FFF|dept2|  7200| high_salary|
|   7| GGG|dept3|  7100| high_salary|
|null|null| null|  7500| high_salary|
|   9| III| null|  4500|  mid_salary|
|  10|null|dept5|  2500|  mid_salary|
+----+----+-----+------+------------+



In [140]:
newdf = df.selectExpr("*", cond)
newdf.show()

+----+----+-----+------+------------+
|  id|name| dept|salary|salary_level|
+----+----+-----+------+------------+
|   1| AAA|dept1|  1000|  low_salary|
|   2| BBB|dept1|  1100|  low_salary|
|   3| CCC|dept1|  3000|  mid_salary|
|   4| DDD|dept1|  1500|  low_salary|
|   5| EEE|dept2|  8000| high_salary|
|   6| FFF|dept2|  7200| high_salary|
|   7| GGG|dept3|  7100| high_salary|
|null|null| null|  7500| high_salary|
|   9| III| null|  4500|  mid_salary|
|  10|null|dept5|  2500|  mid_salary|
+----+----+-----+------+------------+



In [141]:
newdf = df.filter(df["dept"].isNull())
newdf.show()

+----+----+----+------+
|  id|name|dept|salary|
+----+----+----+------+
|null|null|null|  7500|
|   9| III|null|  4500|
+----+----+----+------+



In [142]:
newdf = df.filter(df["dept"].isNotNull())
newdf.show()

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
|  2| BBB|dept1|  1100|
|  3| CCC|dept1|  3000|
|  4| DDD|dept1|  1500|
|  5| EEE|dept2|  8000|
|  6| FFF|dept2|  7200|
|  7| GGG|dept3|  7100|
| 10|null|dept5|  2500|
+---+----+-----+------+



In [143]:
# Replace -1 where the salary is null.
newdf = df.fillna("INVALID", ["dept"])
newdf.show()

+----+----+-------+------+
|  id|name|   dept|salary|
+----+----+-------+------+
|   1| AAA|  dept1|  1000|
|   2| BBB|  dept1|  1100|
|   3| CCC|  dept1|  3000|
|   4| DDD|  dept1|  1500|
|   5| EEE|  dept2|  8000|
|   6| FFF|  dept2|  7200|
|   7| GGG|  dept3|  7100|
|null|null|INVALID|  7500|
|   9| III|INVALID|  4500|
|  10|null|  dept5|  2500|
+----+----+-------+------+



In [ ]:
# borrar los valores nulos del df
# se le pueden pasar parametros como "how". Si how = 'all', elimina los registros cuando todos son nulos

newdf = df.dropna()
newdf.show()

# Ejercicio 1

In [44]:
## Importa el csv de "data/WorldCupPlayers.csv"
## Visualiza los datos

wcPlayers = spark.read.option('header','true').csv('WorldCupPlayers.csv')
wcPlayers.show()

+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|      Player Name|Position|    Event|
+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|      Alex THEPOT|      GK|     null|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|  Oscar BONFIGLIO|      GK|     null|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0| Marcel LANGILLER|    null|     G40'|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Juan CARRENO|    null|     G70'|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|  Ernest LIBERATI|    null|     null|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Rafael GARZA|       C|     null|
|    201| 

In [45]:
## ¿que tipo de datos contiene cada variable?

wcPlayers.dtypes

[('RoundID', 'string'),
 ('MatchID', 'string'),
 ('Team Initials', 'string'),
 ('Coach Name', 'string'),
 ('Line-up', 'string'),
 ('Shirt Number', 'string'),
 ('Player Name', 'string'),
 ('Position', 'string'),
 ('Event', 'string')]

In [46]:
## ¿Cuantos registros hay?

wcPlayers.count()

37784

In [109]:
## Obtén los principales estadísticos de Position

'''wcPlayers.select(
    'Position'
    ).filter(
      'Position is not null'
    ).groupBy(
        'Position'
    ).count().show()'''

wcPlayers.describe('Position').show()

+-------+--------+
|summary|Position|
+-------+--------+
|  count|    4143|
|   mean|    null|
| stddev|    null|
|    min|       C|
|    max|     GKC|
+-------+--------+



In [110]:
## Slecciona y muestra los registros distintos de 'Player Name','Coach Name'

#wcPlayers.groupBy('Player Name','Coach Name').count().show()
wcPlayers.select('Player Name', 'Coach Name').distinct().show()

+--------------------+--------------------+
|         Player Name|          Coach Name|
+--------------------+--------------------+
|    Arturo FERNANDEZ| BRU Francisco (ESP)|
|Cayetano CARRERAS...|DURAND LAGUNA Jos...|
|  Ernesto MASCHERONI|SUPPICI Alberto (...|
|          Aziz FAHMY|   McREA James (SCO)|
|        Gyula POLGAR|    NADAS Odon (HUN)|
|  Ernesto ALBARRACIN|PASCUCCI Felipe (...|
| Armando CASTELLAZZI|POZZO Vittorio (ITA)|
|     Jaroslav BOUCEK|   PETRU Karel (TCH)|
|           Erwin NYC|  KALUZA Jozef (POL)|
|     Stanislaw BARAN|  KALUZA Jozef (POL)|
|     Fernando ROLDAN|BUCCIARDI Arturo ...|
|            Joe MACA|  JEFFREY Bill (SCO)|
|               INDIO|  MOREIRA Zeze (BRA)|
|      Rene DEREUDDRE|PIBAROT Pierre (FRA)|
|    Anton MALATINSKY|    CEJP Josef (TCH)|
|    Alberto MARIOTTI|LORENZO Juan Carl...|
|  Alfredo DI STEFANO|HERRERA Helenio (...|
|             FIDELIS| FEOLA Vicente (BRA)|
|     Stoyan YORDANOV|BOZHKOV Stefan (BUL)|
|      Wim RIJSBERGEN| MICHELS R

In [70]:
## ¿Cuantos partidos con el ID de 1096 ha habido?

wcPlayers.filter(col('MatchID').cast('int') == 1096).count()

33

In [79]:
## Muestra los datos donde la posicion haya sido C y el evento sea G40
# |RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|      Player Name|Position|    Event

wcPlayers.filter(wcPlayers['Position'] == 'C').filter(wcPlayers['Event'].like('G40%')).show()

+-------+-------+-------------+--------------------+-------+------------+----------------+--------+-----+
|RoundID|MatchID|Team Initials|          Coach Name|Line-up|Shirt Number|     Player Name|Position|Event|
+-------+-------+-------------+--------------------+-------+------------+----------------+--------+-----+
|    201|   1089|          PAR|DURAND LAGUNA Jos...|      S|           0|Luis VARGAS PENA|       C| G40'|
|    429|   1175|          HUN|  DIETZ Karoly (HUN)|      S|           0|   Gyorgy SAROSI|       C| G40'|
+-------+-------+-------------+--------------------+-------+------------+----------------+--------+-----+



In [96]:
## Utiliza Spark SQL para mostras los registros donde el MatchID sea mayor o igual a 20

wcPlayers.createOrReplaceTempView("wcPlayers")
spark.sql('select * from wcPlayers where MatchID >= 20').show()

+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|RoundID|MatchID|Team Initials|         Coach Name|Line-up|Shirt Number|      Player Name|Position|    Event|
+-------+-------+-------------+-------------------+-------+------------+-----------------+--------+---------+
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|      Alex THEPOT|      GK|     null|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|  Oscar BONFIGLIO|      GK|     null|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0| Marcel LANGILLER|    null|     G40'|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Juan CARRENO|    null|     G70'|
|    201|   1096|          FRA|CAUDRON Raoul (FRA)|      S|           0|  Ernest LIBERATI|    null|     null|
|    201|   1096|          MEX|   LUQUE Juan (MEX)|      S|           0|     Rafael GARZA|       C|     null|
|    201| 